In [1]:
# NYC Property Data Exploration
from pymongo import MongoClient
import random
import pandas as pd

# Load the dataset
file_path = 'NY_House_Dataset.csv'
df = pd.read_csv(file_path)



# Display the first few rows
df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [2]:
# Load your original DataFrame if not already loaded

# Select the columns you want to retain
columns_to_keep = ['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'SUBLOCALITY', 'LATITUDE', 'LONGITUDE']
cleaned_df = df[columns_to_keep].copy()

# Handle division by zero by replacing 0 with NaN
cleaned_df['BEDS'].replace(0, pd.NA, inplace=True)
cleaned_df['PROPERTYSQFT'].replace(0, pd.NA, inplace=True)

# Calculate PRICE_PER_BED and PRICE_PER_PROPERTYSQFT
cleaned_df['PRICE_PER_BED'] = cleaned_df['PRICE'] / cleaned_df['BEDS']
cleaned_df['PRICE_PER_PROPERTYSQFT'] = cleaned_df['PRICE'] / cleaned_df['PROPERTYSQFT']

# Define the path for the new CSV file
csv_filename = 'filtered_property_data.csv'

# Save the DataFrame to a CSV file
cleaned_df.to_csv(csv_filename, index=False)

# Preview the cleaned dataset to ensure it looks correct
cleaned_df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,SUBLOCALITY,LATITUDE,LONGITUDE,PRICE_PER_BED,PRICE_PER_PROPERTYSQFT
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,Manhattan,40.761255,-73.974483,1.575000e+05,225.000000
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,New York County,40.766393,-73.980991,2.785714e+07,11114.277572
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,Richmond County,40.541805,-74.196109,6.500000e+04,129.032258
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,New York County,40.761398,-73.974613,2.300000e+04,155.056180
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,New York County,40.767224,-73.969856,7.857143e+06,3880.070547


In [3]:
# Constants for MongoDB connection and geospatial calculations
MONGO_URI = 'mongodb://localhost:27017/'
DB_NAME = 'property_database'
COLLECTION_NAME = 'properties'
RADIUS_METERS = 2500

# Constants defining the geographic boundaries of NYC
LAT_MIN = 40.4774
LAT_MAX = 40.9176
LON_MIN = -74.2591
LON_MAX = -73.7004

def insert_data_into_mongodb():
    """
    Load data from CSV and insert it into MongoDB with geospatial indexing.
    """
    # Load your DataFrame if not already loaded
    df = pd.read_csv('filtered_property_data.csv')

    # Combine LATITUDE and LONGITUDE into GeoJSON format
    df['location'] = df.apply(lambda row: {'type': 'Point', 'coordinates': [row['LONGITUDE'], row['LATITUDE']]}, axis=1)

    # Convert the DataFrame to a dictionary format
    data = df.to_dict(orient='records')

    # Establish a connection to MongoDB
    client = MongoClient(MONGO_URI)

    # Specify the database and collection
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # Drop the collection if it exists to start fresh (optional)
    collection.drop()

    # Insert the data into the collection
    collection.insert_many(data)

    # Create a geospatial index on the location field
    collection.create_index([('location', '2dsphere')])

    print("Data has been successfully inserted into MongoDB with geospatial indexing.")

def calculate_average_metrics(center_latitude, center_longitude, radius_meters):
    """
    Calculate the average property price, price per bed, and price per square foot within a specified radius of a given location.

    Parameters:
    center_latitude (float): Latitude of the center point.
    center_longitude (float): Longitude of the center point.
    radius_meters (float): Radius in meters within which to search for properties.

    Returns:
    dict: Average metrics of properties within the radius, or None if no properties are found.
    """
    # Establish a connection to MongoDB
    client = MongoClient(MONGO_URI)
    
    # Specify the database and collection
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # Define the aggregation pipeline
    pipeline = [
        {
            '$geoNear': {
                'near': {'type': 'Point', 'coordinates': [center_longitude, center_latitude]},
                'distanceField': 'distance',
                'spherical': True,
                'maxDistance': radius_meters  # Use the radius in meters directly
            }
        },
        {
            '$group': {
                '_id': None,
                'average_price': {'$avg': '$PRICE'},
                'average_price_per_bed': {'$avg': '$PRICE_PER_BED'},
                'average_price_per_sqft': {'$avg': '$PRICE_PER_PROPERTYSQFT'}
            }
        }
    ]

    # Execute the aggregation
    result = list(collection.aggregate(pipeline))

    # Return the average metrics if result is found, else return None
    if result:
        return result[0]
    else:
        return None

def main():
    # Insert data into MongoDB (this step should be done once or when data updates)
    insert_data_into_mongodb()

    lng = 40.697359
    lat = -73.931822
    metrics = calculate_average_metrics(lng, lat, RADIUS_METERS)
    if metrics:
        print(f"Metrics within {RADIUS_METERS} meters of ({lat:.6f}, {lng:.6f}): Average price: ${metrics['average_price']:.2f} Average price per bed: ${metrics['average_price_per_bed']:.2f} Average price per square foot: ${metrics['average_price_per_sqft']:.2f}")
    else:
        print(f"No properties found within {RADIUS_METERS} meters of ({lat:.6f}, {lng:.6f}).")
        

if __name__ == "__main__":
    main()


Data has been successfully inserted into MongoDB with geospatial indexing.
Metrics within 2500 meters of (-73.931822, 40.697359): Average price: $1673757.29 Average price per bed: $475885.07 Average price per square foot: $732.14
